In [ ]:
!git clone https://github.com/pie3636/newsjam.git
!mv newsjam/* .

Cloning into 'newsjam'...
remote: Enumerating objects: 387, done.
remote: Counting objects: 100% (387/387), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 387 (delta 206), reused 246 (delta 108), pack-reused 0
Receiving objects: 100% (387/387), 9.33 MiB | 4.67 MiB/s, done.
Resolving deltas: 100% (206/206), done.


Imports (Run Once)

In [ ]:
!pip install selenium --upgrade
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!python -m pip install -r requirements.txt
!python -m spacy download fr_core_news_sm

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from itertools import groupby
from selenium.webdriver.chrome.options import Options
import re
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') #make sure the webdriver is in the right spot

def actu_scraper(url):
    '''
    takes a given URL, grabs the article text
    returns the text as a string
    '''

    #webdriver initialization stuff
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get(url)

    #bypasses the cookie popup and grabs the article
    try:
      button = driver.find_element(By.CSS_SELECTOR, "#didomi-notice-agree-button")
      button.click()
    except:
      article = driver.find_elements(By.XPATH, '/html/body/div[2]/main/div/div[1]/div[1]')
    article = driver.find_elements(By.XPATH, '/html/body/div[2]/main/div/div[1]/div[1]')
    article_text = []
    #article text cleanup, automatically removes all image captions, extraneous information at the start and end of every article, etc.
    for x in range(len(article)):
        article_text.append(article[x].text.replace("\n"," "))
        article_text[x] = re.sub(r'Par.*\d{1,2}:\d{1,2}|Cet article vous a été.*|\(©.*?\)|<a.*?a>',"",article_text[x])
    driver.close()
    return article_text[x]

text = actu_scraper('https://actu.fr/bretagne/vitre_35360/vitre-comment-prendre-rendez-vous-pour-vacciner-les-enfants-de-5-11-ans_47661576.html')

In [16]:
from summ.lsa import LSASummarizer
lsa_summ = LSASummarizer()
summary = lsa_summ.get_summary(text)
summary = summary[0]

In [17]:
summary = summary.replace('\n', ' ')
print(summary)

Le Journal de Vitré Mon actu Suivre Depuis le 22 décembre 2021, la vaccination contre la Covid-19 est ouverte à l’ensemble des enfants âgés de 5 à 11 ans.
